In [ ]:
# ==============================================================
# MNIST PIPELINE
# ==============================================================

from kfp import dsl
from kfp.dsl import component, pipeline, OutputPath, InputPath

BASE_IMAGE = "python:3.11-slim"

In [ ]:
# 1. 데이터 다운로드 (이미 있으면 스킵!)
@component(
    base_image=BASE_IMAGE,
    packages_to_install=["tensorflow==2.16.1", "numpy"]
)
def download_mnist_if_needed(dataset_out: OutputPath("Dataset")):
    import os
    import tensorflow as tf
    import numpy as np
    
    target_file = os.path.join(dataset_out, "mnist.npz")
    
    # 이미 있으면 패스! (Kubeflow 캐시 자동 활용)
    if os.path.exists(target_file):
        print("MNIST 데이터 이미 존재 → 다운로드 스킵!")
        return
    
    print("MNIST 데이터 다운로드 시작...")
    os.makedirs(dataset_out, exist_ok=True)
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test  = x_test.astype("float32")  / 255.0
    x_train = np.expand_dims(x_train, -1)
    x_test  = np.expand_dims(x_test, -1)
    
    np.savez(target_file,
             x_train=x_train, y_train=y_train,
             x_test=x_test, y_test=y_test)
    
    print(f"MNIST 저장 완료: {target_file}")

In [ ]:
# 2. 학습
@component(
    base_image=BASE_IMAGE,
    packages_to_install=["tensorflow==2.16.1"]
)
def train_mnist(dataset_in: InputPath("Dataset"), model_out: OutputPath("Model")):
    import os, numpy as np, tensorflow as tf
    
    data = np.load(os.path.join(dataset_in, "mnist.npz"))
    x_train, y_train = data["x_train"], data["y_train"]
    
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(), 
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=5, batch_size=128, verbose=2)
    
    save_path = os.path.join(model_out, "model.keras")
    os.makedirs(model_out, exist_ok=True)
    model.save(save_path)
    print(f"모델 학습 & 저장 완료: {save_path}")

In [ ]:
# 3. 평가
@component(
    base_image=BASE_IMAGE,
    packages_to_install=["tensorflow==2.16.1"]
)
def evaluate_mnist(model_in: InputPath("Model"), dataset_in: InputPath("Dataset")):
    import os, numpy as np, tensorflow as tf
    
    model = tf.keras.models.load_model(os.path.join(model_in, "model.keras"))
    data = np.load(os.path.join(dataset_in, "mn                      ist.npz"))
    x_test, y_test = data["x_test"], data["y_test"]
    
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"최종 테스트 정확도: {acc:.4f} (loss: {loss:.4f})")
    print("파이프라인 완전 성공!")

In [ ]:
# 4. 파이프라인 (진짜 심플!)
@pipeline(name="MNIST - 심플 & 확실한 파이프라인")
def mnist_pipeline():
    download = download_mnist_if_needed()
    
    train = train_mnist(dataset_in=download.outputs["dataset_out"])
    
    evaluate = evaluate_mnist(
        model_in=train.outputs["model_out"],
        dataset_in=download.outputs["dataset_out"]
    )

print("파이프라인 준비 완료!")

In [ ]:
import os
os.environ["KUBEFLOW_HOST"] = "https://대시보드 접속한 로드밸런서의 Public IP/"
os.environ["KUBEFLOW_USERNAME"] = "Kubeflow 대시보드 접속 아이디(이메일)"
os.environ["KUBEFLOW_PASSWORD"] = "Kubeflow 대시보드 접속 비밀번호"

# 2. 클라이언트 생성 & 실행
from kakaocloud_kbm import KbmPipelineClient

client = KbmPipelineClient(verify_ssl=False)
client.create_run_from_pipeline_func(
    mnist_pipeline,
    experiment_name="MNIST 심플",
    run_name="파이프라인_실행"
)

print("파이프라인 실행 완료! 이제 대시보드에서 확인하세요!")